In [1]:
def txt_to_list(path):
    with open(path, 'r') as f:
        txt = str(f.read())
    txt_list = txt.split('\n')
    text_label_list = list()
    tmp = list()
    for line in txt_list:
        if line == '--------------------':
            text_label_list.append(tmp)
            tmp = list()
            continue
        if line == '':
            continue
        tmp.append(line)
    x = list()
    y = list()
    for text_label in text_label_list:
        text = text_label[0]
        text_list = [i for i in text]
        label = text_label[2:]
        label_list = ['O' for i in range(len(text_list))]
        for i in label:
            entity = i.split('\t')
            b = int(entity[1])
            label_list[b] = 'B-{}'.format(entity[-1])
            for j in range(int(entity[1])+1, int(entity[2])):
                label_list[j] = 'I-{}'.format(entity[-1])
        while len(text_list) > 32:            
            x.append(text_list[:32])
            y.append(label_list[:32])
            text_list = text_list[32:]
            label_list = label_list[32:]
        x.append(text_list)
        y.append(label_list)
    return [x, y]

In [2]:
x1, y1 = txt_to_list('../datasets/stage1/SampleData_deid.txt')

In [3]:
x2, y2 = txt_to_list('../datasets/stage2/train_1_update.txt')

In [4]:
x4, y4 = txt_to_list('../datasets/stage4/train_2.txt')

In [5]:
x = x1 + x2 + x4
len(x)

22532

In [6]:
y = y1 + y2 + y4
len(y)

22532

In [7]:
# print(x[0])

In [8]:
# print(y[0])

In [9]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, random_state=42)
train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)
print(len(train_x), len(train_y))
print(len(valid_x), len(valid_y))
print(len(test_x), len(test_y))

14420 14420
3605 3605
4507 4507


In [120]:
# import kashgari
# kashgari.config.use_cudnn_cell = True
# from kashgari.tasks.labeling.models import BiLSTM_Model

# model = BiLSTM_Model()
# model.fit(train_x,
#           train_y,
#           valid_x,
#           valid_y,
#           epochs=10,
#          )
# model.evaluate(test_x, test_y)
# model.save('model/BiLSTM')

In [11]:
# loaded_model = kashgari.utils.load_model('model/BiLSTM')
# loaded_model.predict(test_x)[0]

In [121]:
import kashgari
from kashgari.tasks.labeling.models import BiLSTM_CRF_Model
from kashgari.embeddings import BERTEmbedding

bert_embed = BERTEmbedding('chinese_L-12_H-768_A-12',
                           task=kashgari.LABELING,
                           sequence_length=32,
                          )

model = BiLSTM_CRF_Model(bert_embed)
model = BiLSTM_CRF_Model()
model.fit(train_x,
          train_y,
          valid_x,
          valid_y,
          epochs=10,
         )

model.evaluate(test_x, test_y)
model.save('model/BiLSTM_CRF_Model')

Model: "model_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 32)]              0         
_________________________________________________________________
layer_embedding (Embedding)  (None, 32, 100)           173600    
_________________________________________________________________
layer_blstm (Bidirectional)  (None, 32, 256)           235520    
_________________________________________________________________
layer_dense (Dense)          (None, 32, 64)            16448     
_________________________________________________________________
layer_crf_dense (Dense)      (None, 32, 27)            1755      
_________________________________________________________________
layer_crf (CRF)              (None, 32, 27)            729       
Total params: 428,052
Trainable params: 428,052
Non-trainable params: 0
____________________________________________________

In [122]:
import kashgari
from kashgari.tasks.labeling import BiLSTM_Model
from kashgari.embeddings import BERTEmbedding

bert_embed = BERTEmbedding('chinese_L-12_H-768_A-12',
                           task=kashgari.LABELING,
                           sequence_length=32,
                          )

model = BiLSTM_Model(bert_embed)
model = BiLSTM_Model()
model.fit(train_x,
          train_y,
          valid_x,
          valid_y,
          epochs=10,
         )

model.evaluate(test_x, test_y)
model.save('model/BiLSTM_Model')

Model: "model_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 32)]              0         
_________________________________________________________________
layer_embedding (Embedding)  (None, 32, 100)           173600    
_________________________________________________________________
layer_blstm (Bidirectional)  (None, 32, 256)           235520    
_________________________________________________________________
layer_dropout (Dropout)      (None, 32, 256)           0         
_________________________________________________________________
layer_time_distributed (Time (None, 32, 27)            6939      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 27)            0         
Total params: 416,059
Trainable params: 416,059
Non-trainable params: 0
____________________________________________________

In [123]:
model = kashgari.utils.load_model('model/BiLSTM_CRF_Model')

In [124]:
import pandas as pd
df = pd.read_csv('../datasets/stage4/development_2.csv')

In [125]:
def predict_NER(model, text, sequence_length):
    x_list = list()
    while len(text) > sequence_length:
        x_list.append([i for i in text[:sequence_length]])
        text = text[sequence_length:]        
    x_list.append([i for i in text[:sequence_length]])
    y_list = model.predict(x_list)
    merge_list = list()
    for i in y_list:
        for j in i:
            merge_list.append(j)
    return merge_list   

In [126]:
def output_NER(article_id, text, y):
    output = str()
    flag = False
    for i, j in enumerate(y):
        if j != 'O':
            if j[0] == 'B':
                start_position = i
                entity_type = j.split('-')[-1]
                flag = True
        elif j == 'O' and flag == True:
            end_position = i
            flag = False
            entity_text = text[start_position: end_position]
            output += '{}\t{}\t{}\t{}\t{}\n'.format(article_id, start_position, end_position, entity_text, entity_type)  
    return output

In [117]:
output="article_id\tstart_position\tend_position\tentity_text\tentity_type\n"
for article_id, text in zip(df['article_id'], df['text']):
    y = predict_NER(model, text, 32)
    output += output_NER(article_id, text, y)    

In [118]:
print(output)

article_id	start_position	end_position	entity_text	entity_type
0	273	275	李醫	name
0	320	322	25	med_exam
0	326	328	25	med_exam
0	615	619	11點多	time
0	1168	1172	7月21	time
0	1571	1572	1	med_exam
0	1697	1699	第二	time
0	1707	1709	第二	time
0	2200	2203	13號	time
0	2214	2215	1	time
0	2226	2228	3天	time
0	2319	2321	禮拜	time
0	2332	2334	禮拜	time
0	2738	2740	林區	location
0	2807	2810	下禮拜	time
0	3306	3308	3天	time
0	3340	3343	下禮拜	time
1	59	62	108	med_exam
1	66	69	108	med_exam
1	457	460	五月中	time
2	27	32	8月17號	time
2	40	45	上上個禮拜	time
2	53	56	110	med_exam
2	85	88	110	med_exam
2	111	114	6.0	med_exam
2	253	256	126	time
2	279	281	2年	time
2	496	498	泰國	location
2	524	526	釣蝦	name
2	1114	1116	半年	time
2	1118	1120	半年	time
2	1147	1149	三個	time
2	1227	1231	兩個禮拜	time
3	183	185	7點	time
3	233	236	9點多	time
3	250	255	6月15號	time
3	256	260	16號左	time
3	265	268	16號	time
3	272	274	14	time
3	278	281	16號	time
3	302	306	1個禮拜	time
3	576	580	6月4號	time
3	582	584	前年	time
3	592	594	前年	time
3	1431	1433	禮拜	time
3	1443	1446	禮拜2	time
3	1456	145

In [119]:
output_path='bert32.tsv'
with open(output_path, 'w', encoding='utf-8') as f:
    f.write(output)